In [1]:
!pip -q install ccxt yfinance arch scikit-learn tensorflow python-binance pandas numpy
#install library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.3/981.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.0/144.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 641.1/641.1 kB 21.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error


np.random.seed(42)
tf.random.set_seed(42)



In [3]:

def fetch_yahoo_ohlc(symbol="BTC-USD", start="2017-01-01"):
    import yfinance as yf
    import pandas as pd

    df = yf.download(symbol, start=start, auto_adjust=False, progress=False)

    # 1) flatten MultiIndex columns if present
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    # 2) remove the columns' name (often shows as "Price")
    df.columns.name = None

    # 3) standardize column names
    df = df.rename(columns={
        "Open": "open",
        "High": "high",
        "Low": "low",
        "Close": "close",
        "Volume": "volume",
    })

    df.index = pd.to_datetime(df.index)
    df = df[["open", "high", "low", "close", "volume"]].dropna().sort_index()
    df["source"] = "yahoo_finance"
    return df


def get_btc_daily(start="2017-01-01"):
    return fetch_yahoo_ohlc("BTC-USD", start)

df = get_btc_daily("2017-01-01")
df.tail()


,open,high,low,close,volume,source
Date,,,,,,
2025-12-17,87847.617188,90264.570312,85316.265625,86143.757812,44243392914,yahoo_finance
2025-12-18,86144.367188,89412.664062,84436.312500,85462.507812,52667115348,yahoo_finance
2025-12-19,85476.132812,89339.117188,85107.664062,88103.382812,46733310561,yahoo_finance
2025-12-20,88101.671875,88497.203125,87924.875000,88344.000000,14688196659,yahoo_finance
2025-12-22,88621.664062,89396.585938,88168.570312,88929.164062,24414984192,yahoo_finance


In [4]:
# 1) log return
df["log_ret"] = np.log(df["close"]).diff()

# 2) Range-based variance proxies
hl = np.log(df["high"] / df["low"]).replace([np.inf, -np.inf], np.nan)
co = np.log(df["close"] / df["open"]).replace([np.inf, -np.inf], np.nan)

# Parkinson (variance)
df["rv_parkinson"] = (hl**2) / (4.0 * np.log(2.0))

# Garman–Klass (variance)
df["rv_gk"] = 0.5 * (hl**2) - (2.0 * np.log(2.0) - 1.0) * (co**2)
df["rv_gk"] = df["rv_gk"].clip(lower=0)  # avoid tiny negative due to numeric issues

# 3) HAR features (use rv_gk as main volatility proxy)
x = df["rv_gk"]
df["RV_d"] = x.shift(1)
df["RV_w"] = x.shift(1).rolling(7).mean()
df["RV_m"] = x.shift(1).rolling(30).mean()

# 4) Extra features (lightweight, helpful)
df["log_volume"] = np.log(df["volume"].replace(0, np.nan))
df["abs_log_ret"] = df["log_ret"].abs()

# 5) Target: 7-day ahead average realized variance (future)
df["y_7d"] = df["rv_gk"].shift(-1).rolling(7).mean()

# 6) Modeling dataframe
features = ["RV_d", "RV_w", "RV_m", "log_volume", "abs_log_ret"]
target = "y_7d"
data = df[features + [target]].dropna().copy()

print("data shape:", data.shape)
print("date range:", data.index.min().date(), "to", data.index.max().date())
data.head()


data shape: (3246, 6)
date range: 2017-01-31 to 2025-12-20


,RV_d,RV_w,RV_m,log_volume,abs_log_ret,y_7d
Date,,,,,,
2017-01-31,0.000008,0.000089,0.002247,18.918919,0.052923,0.000081
2017-02-01,0.000399,0.000080,0.002243,18.826879,0.019006,0.000111
2017-02-02,0.000044,0.000081,0.002231,18.797890,0.022769,0.000131
2017-02-03,0.000277,0.000111,0.002239,19.120198,0.017741,0.000187
2017-02-04,0.000175,0.000131,0.002192,18.859348,0.012534,0.000203


In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

features = ["RV_d", "RV_w", "RV_m", "log_volume", "abs_log_ret"]
target = "y_7d"

# splite
split_date = "2022-01-01"
train = data.loc[data.index < split_date].copy()
test  = data.loc[data.index >= split_date].copy()

X_train, y_train = train[features].values, train[target].values
X_test,  y_test  = test[features].values,  test[target].values

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_s, y_train)
pred = model.predict(X_test_s)

rmse = np.sqrt(mean_squared_error(y_test, pred))
mae  = mean_absolute_error(y_test, pred)
# qlike用训练期 y 的一个下界（比如 1% 分位数）
lower_bound = np.quantile(y_train, 0.01)

pred_safe = np.maximum(pred, lower_bound)
y_safe = np.maximum(y_test, lower_bound)

qlike = np.mean((y_safe / pred_safe) - np.log(y_safe / pred_safe) - 1.0)

print("HAR baseline | RMSE:", rmse, "| MAE:", mae, "| QLIKE:", qlike)


HAR baseline | RMSE: 0.0003103086023304309 | MAE: 0.00020356836706419593 | QLIKE: 0.09001479398106904


In [6]:
!pip -q install arch


In [7]:
from arch import arch_model
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 1) 准备 returns（和你 df 对齐）
# 用全量 df 的 log_ret，但对齐到 data 的 index，避免日期不一致
rets = df["log_ret"].loc[data.index].dropna()

# 对齐 y_test 的日期范围
split_date = "2022-01-01"
test_idx = data.loc[data.index >= split_date].index
y_test_garch = data.loc[test_idx, "y_7d"].values

# 2) 训练样本 returns（仅用 split_date 之前）
rets_train = rets.loc[rets.index < split_date] * 100

# 3) 拟合 GARCH(1,1)
am = arch_model(rets_train, vol="GARCH", p=1, q=1, mean="Zero", dist="normal")
res = am.fit(disp="off")

# 4) 对测试集做滚动预测（walk-forward）
# 每天用截至当天的数据重新 forecast
h = 7
pred_list = []
current_returns = rets.loc[rets.index < split_date].copy()

for t in test_idx:
    # 拿到截至 t-1 的 returns
    current_returns = rets.loc[rets.index < t] * 100
    am_t = arch_model(current_returns, vol="GARCH", p=1, q=1, mean="Zero", dist="normal")
    res_t = am_t.fit(disp="off")

    fc = res_t.forecast(horizon=h, reindex=False)
    # fc.variance 是未来 1..h 天的方差预测（单位是 returns*100 的方差）
    var_h = fc.variance.values[-1, :] / (100**2)  # 还原到原始 returns 的方差
    pred_7d = np.mean(var_h)  # 取未来7天均值
    pred_list.append(pred_7d)

pred_garch = np.array(pred_list)

rmse_g = np.sqrt(mean_squared_error(y_test_garch, pred_garch))
mae_g = mean_absolute_error(y_test_garch, pred_garch)

eps = 1e-12
pred_safe = np.maximum(pred_garch, eps)
y_safe = np.maximum(y_test_garch, eps)
qlike_g = np.mean((y_safe / pred_safe) - np.log(y_safe / pred_safe) - 1.0)

print("GARCH(1,1) | RMSE:", rmse_g, "| MAE:", mae_g, "| QLIKE:", qlike_g)


GARCH(1,1) | RMSE: 0.000636590105583877 | MAE: 0.0005712124207374092 | QLIKE: 0.3118591241311017


LSTM


In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf


features = ["RV_d", "RV_w", "RV_m", "log_volume", "abs_log_ret"]
target = "y_7d"
split_date = "2022-01-01"

train_df = data.loc[data.index < split_date].copy()
test_df  = data.loc[data.index >= split_date].copy()

# 标准化特征（只对 features；target 不标准化，保持可解释性）
scaler = StandardScaler()
train_df[features] = scaler.fit_transform(train_df[features])
test_df[features]  = scaler.transform(test_df[features])

print("train rows:", len(train_df), "test rows:", len(test_df))
train_df.head()


train rows: 1796 test rows: 1450


,RV_d,RV_w,RV_m,log_volume,abs_log_ret,y_7d
Date,,,,,,
2017-01-31,-0.447578,-0.679922,0.435382,-2.928291,0.787116,0.000081
2017-02-01,-0.335680,-0.683993,0.432305,-2.992445,-0.303168,0.000111
2017-02-02,-0.437088,-0.683623,0.424977,-3.012651,-0.182219,0.000131
2017-02-03,-0.370527,-0.669973,0.429734,-2.787995,-0.343852,0.000187
2017-02-04,-0.399661,-0.660553,0.399698,-2.969813,-0.511226,0.000203


In [9]:
seq_len = 22

def make_sequences(df, feature_cols, target_col, seq_len):
    X, y, idx = [], [], []
    vals_x = df[feature_cols].values.astype(np.float32)
    vals_y = df[target_col].values.astype(np.float32)
    dates = df.index

    for i in range(seq_len, len(df)):
        X.append(vals_x[i-seq_len:i, :])  # 过去 seq_len 天
        y.append(vals_y[i])               # 当天的 y_7d
        idx.append(dates[i])

    return np.array(X), np.array(y), np.array(idx)

X_train, y_train, idx_train = make_sequences(train_df, features, target, seq_len)
X_test,  y_test,  idx_test  = make_sequences(test_df,  features, target, seq_len)

print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_test :", X_test.shape,  "y_test :", y_test.shape)
print("first test date:", idx_test[0], "last test date:", idx_test[-1])


X_train: (1774, 22, 5) y_train: (1774,)
X_test : (1428, 22, 5) y_test : (1428,)
first test date: 2022-01-23 00:00:00 last test date: 2025-12-20 00:00:00


In [11]:
from tensorflow.keras import layers, callbacks, models

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(seq_len, len(features))),
    tf.keras.layers.LSTM(16),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(1, activation="softplus")
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mse")

es = callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[es],
    verbose=1
)

pred = model.predict(X_test, verbose=0).reshape(-1)

rmse = np.sqrt(mean_squared_error(y_test, pred))
mae  = mean_absolute_error(y_test, pred)

# QLIKE（和 HAR 同规则：用训练期 y 的 1% 分位数做下界）
lower_bound = np.quantile(y_train, 0.01)
pred_safe = np.maximum(pred, lower_bound)
y_safe = np.maximum(y_test, lower_bound)
qlike = np.mean((y_safe / pred_safe) - np.log(y_safe / pred_safe) - 1.0)

print("LSTM | RMSE:", rmse, "| MAE:", mae, "| QLIKE:", qlike)



Epoch 1/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.4007 - val_loss: 0.2677
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.1492 - val_loss: 0.1366
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0335 - val_loss: 0.0504
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0055 - val_loss: 0.0123
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0020 - val_loss: 0.0047
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0011 - val_loss: 0.0024
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 6.8098e-04 - val_loss: 0.0014
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 4.6327e-04 - val_loss: 9.4900e-04
Epoch 9/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 3.3583e-04 - val_loss: 6.7367e-04
Epoch 10/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 2.5690e-04 - val_loss: 5.0187e-04
Epoch 11/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 2.0411e-04 - val_loss: 3.8720e-04
Epoch 12/50
45/45 ━━━━━━━━━

In [12]:
import numpy as np
import pandas as pd
import yfinance as yf

def fetch_yahoo_ohlcv(symbol="BTC-USD", start="2017-01-01"):
    df = yf.download(symbol, start=start, auto_adjust=False, progress=False)
    df.index = pd.to_datetime(df.index)
    df = df.rename(columns={
        "Open": "open", "High": "high", "Low": "low",
        "Close": "close", "Adj Close": "adj_close", "Volume": "volume"
    })
    # 有些时候会出现多层列名（Price/Ticker），做一次扁平化兜底
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [c[0].lower() if isinstance(c, tuple) else str(c).lower() for c in df.columns]

    df = df.sort_index()
    return df

def add_macd(df, price_col="close", fast=12, slow=26, signal=9):
    # EMA
    ema_fast = df[price_col].ewm(span=fast, adjust=False).mean()
    ema_slow = df[price_col].ewm(span=slow, adjust=False).mean()

    df["macd"] = ema_fast - ema_slow
    df["macd_signal"] = df["macd"].ewm(span=signal, adjust=False).mean()
    df["macd_hist"] = df["macd"] - df["macd_signal"]

    return df

def add_volume_features(df):
    # log volume（你已有）
    df["log_volume"] = np.log(df["volume"].replace(0, np.nan))

    # 成交量变化（非线性很常用）
    df["vol_chg"] = df["volume"].pct_change()

    # 成交量相对强度：相对过去30天均值的偏离
    df["vol_z30"] = (df["volume"] - df["volume"].rolling(30).mean()) / df["volume"].rolling(30).std()

    return df

btc = fetch_yahoo_ohlcv("BTC-USD", start="2017-01-01")
btc = add_macd(btc, price_col="close", fast=12, slow=26, signal=9)
btc = add_volume_features(btc)

btc[["close","volume","macd","macd_signal","macd_hist","log_volume","vol_chg","vol_z30"]].tail()


,close,volume,macd,macd_signal,macd_hist,log_volume,vol_chg,vol_z30
Date,,,,,,,,
2025-12-17,86143.757812,44243392914,-1804.848263,-1734.776024,-70.072239,24.512972,0.072251,-0.981022
2025-12-18,85462.507812,52667115348,-1958.084183,-1779.437656,-178.646527,24.687257,0.190395,-0.529488
2025-12-19,88103.382812,46733310561,-1845.158374,-1792.581799,-52.576575,24.567723,-0.112666,-0.772861
2025-12-20,88344.000000,14688196659,-1716.461758,-1777.357791,60.896033,23.410310,-0.685702,-2.176287
2025-12-22,88811.289062,24467017728,-1558.793639,-1733.644961,174.851322,23.920592,0.665760,-1.868450


In [13]:
df = add_macd(df, "close")
df = add_volume_features(df)

extra = ["macd","macd_signal","macd_hist","vol_chg","vol_z30"]
data_aug = df[features + extra + [target]].dropna().copy()

data_aug

,RV_d,RV_w,RV_m,log_volume,abs_log_ret,macd,macd_signal,macd_hist,vol_chg,vol_z30,y_7d
Date,,,,,,,,,,,
2017-01-31,0.000008,0.000089,0.002247,18.918919,0.052923,0.705035,-9.732173,10.437208,1.103895,-0.018826,0.000081
2017-02-01,0.000399,0.000080,0.002243,18.826879,0.019006,6.184670,-6.548804,12.733475,-0.087932,-0.135529,0.000111
2017-02-02,0.000044,0.000081,0.002231,18.797890,0.022769,12.224319,-2.794180,15.018498,-0.028572,-0.164339,0.000131
2017-02-03,0.000277,0.000111,0.002239,19.120198,0.017741,18.261604,1.416977,16.844627,0.380309,0.441547,0.000187
2017-02-04,0.000175,0.000131,0.002192,18.859348,0.012534,23.819800,5.897542,17.922259,-0.229603,0.123299,0.000203
...,...,...,...,...,...,...,...,...,...,...,...
2025-12-16,0.001271,0.000684,0.000782,24.443212,0.016346,-1652.374805,-1717.257964,64.883160,-0.094324,-1.185977,0.000702
2025-12-17,0.000414,0.000559,0.000774,24.512972,0.019545,-1804.848263,-1734.776024,-70.072239,0.072251,-0.981022,0.000782
2025-12-18,0.001441,0.000702,0.000787,24.687257,0.007940,-1958.084183,-1779.437656,-178.646527,0.190395,-0.529488,0.000844


define dataset

In [14]:
features_aug = [
    "RV_d", "RV_w", "RV_m", "log_volume", "abs_log_ret",
    "macd", "macd_signal", "macd_hist", "vol_chg", "vol_z30"
]
target = "y_7d"

In [15]:
split_date = "2022-01-01"

train_df = data_aug.loc[data_aug.index < split_date].copy()
test_df  = data_aug.loc[data_aug.index >= split_date].copy()

print("train rows:", len(train_df), "test rows:", len(test_df))

train rows: 1796 test rows: 1450


In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train_df[features_aug] = scaler.fit_transform(train_df[features_aug])
test_df[features_aug]  = scaler.transform(test_df[features_aug])


In [17]:
seq_len = 22

def make_sequences(df, feature_cols, target_col, seq_len):
    X, y, idx = [], [], []

    vals_x = df[feature_cols].values.astype("float32")
    vals_y = df[target_col].values.astype("float32")
    dates  = df.index

    for i in range(seq_len, len(df)):
        X.append(vals_x[i-seq_len:i, :])  # (seq_len, n_features)
        y.append(vals_y[i])
        idx.append(dates[i])

    return np.array(X), np.array(y), np.array(idx)


In [18]:
X_train, y_train, idx_train = make_sequences(
    train_df, features_aug, target, seq_len
)

X_test, y_test, idx_test = make_sequences(
    test_df, features_aug, target, seq_len
)

print("X_train:", X_train.shape)
print("X_test :", X_test.shape)


X_train: (1774, 22, 10)
X_test : (1428, 22, 10)


In [19]:
import tensorflow as tf
from tensorflow.keras import layers, callbacks

model = tf.keras.Sequential([
    layers.Input(shape=(seq_len, len(features_aug))),
    layers.LSTM(16),
    layers.Dense(8, activation="relu"),
    layers.Dense(1, activation="softplus")  # 波动率非负
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="mse"
)

es = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[es],
    verbose=1
)

pred = model.predict(X_test, verbose=0).reshape(-1)

rmse = np.sqrt(mean_squared_error(y_test, pred))
mae  = mean_absolute_error(y_test, pred)

# QLIKE：与 HAR 完全一致
lower_bound = np.quantile(y_train, 0.01)
pred_safe = np.maximum(pred, lower_bound)
y_safe    = np.maximum(y_test, lower_bound)

qlike = np.mean(y_safe / pred_safe - np.log(y_safe / pred_safe) - 1.0)

print("LSTM+MACD+VOL | RMSE:", rmse, "| MAE:", mae, "| QLIKE:", qlike)



Epoch 1/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.2952 - val_loss: 0.1948
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0792 - val_loss: 0.1197
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0193 - val_loss: 0.0793
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0082 - val_loss: 0.0604
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0042 - val_loss: 0.0484
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0024 - val_loss: 0.0405
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0015 - val_loss: 0.0345
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 9.9942e-04 - val_loss: 0.0299
Epoch 9/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 7.2201e-04 - val_loss: 0.0263
Epoch 10/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 5.4486e-04 - val_loss: 0.0235
Epoch 11/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 4.2228e-04 - val_loss: 0.0212
Epoch 12/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/

new test


In [20]:
split_date = "2022-01-01"
train_df = data_aug.loc[data_aug.index < split_date].copy()
test_df  = data_aug.loc[data_aug.index >= split_date].copy()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_df[features_aug] = scaler.fit_transform(train_df[features_aug])
test_df[features_aug]  = scaler.transform(test_df[features_aug])


In [21]:
seq_len = 22

X_train, y_train, idx_train = make_sequences(train_df, features_aug, target, seq_len)
X_test,  y_test,  idx_test  = make_sequences(test_df,  features_aug, target, seq_len)

print(X_train.shape, X_test.shape)  # 应该是 (样本数, 22, 10)


(1774, 22, 10) (1428, 22, 10)


In [22]:
import numpy as np

n = len(X_train)
cut = int(n * 0.8)
X_tr, y_tr = X_train[:cut], y_train[:cut]
X_va, y_va = X_train[cut:], y_train[cut:]


In [26]:
import tensorflow as tf
from tensorflow.keras import layers, callbacks

model = tf.keras.Sequential([
    layers.Input(shape=(seq_len, len(features_aug))),
    layers.LSTM(16),
    layers.Dense(8, activation="relu"),
    layers.Dense(1, activation="softplus")  # 保证预测为正
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mse")

es = callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(
    X_tr, y_tr,
    validation_data=(X_va, y_va),
    epochs=50,
    batch_size=32,
    shuffle=False,   # 关键：时间序列别 shuffle
    callbacks=[es],
    verbose=1
)


Epoch 1/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.5977 - val_loss: 0.5385
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.3391 - val_loss: 0.4354
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.2024 - val_loss: 0.3646
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.1053 - val_loss: 0.2956
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0462 - val_loss: 0.1998
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0212 - val_loss: 0.1081
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0115 - val_loss: 0.0748
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0070 - val_loss: 0.0550
Epoch 9/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0046 - val_loss: 0.0423
Epoch 10/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0033 - val_loss: 0.0337
Epoch 11/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0024 - val_loss: 0.0276
Epoch 12/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0

In [27]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

pred = model.predict(X_test, verbose=0).reshape(-1)
rmse = np.sqrt(mean_squared_error(y_test, pred))
mae  = mean_absolute_error(y_test, pred)

lower_bound = np.quantile(y_tr, 0.01)  # 用训练期 y 的 1% 分位做下界（你现在的严谨版规则）
pred_safe = np.maximum(pred, lower_bound)
y_safe    = np.maximum(y_test, lower_bound)
qlike = np.mean((y_safe / pred_safe) - np.log(y_safe / pred_safe) - 1.0)

print("LSTM+AUG | RMSE:", rmse, "| MAE:", mae, "| QLIKE:", qlike)


LSTM+AUG | RMSE: 0.040357601936136925 | MAE: 0.019023213535547256 | QLIKE: 2.1167874


In [ ]:
print("y_train raw min/mean/max:", train_df[target].min(), train_df[target].mean(), train_df[target].max())
print("y_test  raw min/mean/max:",  test_df[target].min(),  test_df[target].mean(),  test_df[target].max())
print("example y_train head:", train_df[target].head())


y_train raw min/mean/max: 4.00214356059557e-05 0.00157465884045342 0.017389304738662233
y_test  raw min/mean/max: 4.303581341398371e-05 0.0006775936689352785 0.005557682311444191
example y_train head: Date
2017-01-31    0.000081
2017-02-01    0.000111
2017-02-02    0.000131
2017-02-03    0.000187
2017-02-04    0.000203
Name: y_7d, dtype: float64


In [ ]:
pred = model.predict(X_test, verbose=0).reshape(-1)

print("y_test min/mean/max:", y_test.min(), y_test.mean(), y_test.max())
print("pred   min/mean/max:", pred.min(), pred.mean(), pred.max())

print("pred head:", pred[:10])
print("y_test head:", y_test[:10])


y_test min/mean/max: 4.3035812e-05 0.0006758111 0.0055576824
pred   min/mean/max: -8.099101 -7.070105 -3.0086966
pred head: [-6.7047963 -6.558874  -6.322838  -6.2118363 -6.163665  -6.274892
 -6.5132146 -6.9978905 -7.3787556 -7.6007185]
y_test head: [0.00230288 0.0023946  0.00264796 0.00250205 0.00196416 0.00180659
 0.00175122 0.00097628 0.00087764 0.00065093]
